本笔记本记录{mlogi}包的学习笔记。

# 数据管理

本节思路很简单，了解我们需要的输入数据格式，然后指定转化思路，获得我们想要的数据。

## 输入数据

我们想要的数据，需要有个体标识（case），方案标识（alt），选择标识（choice，表示是否选择），其余是预测变量。

In [1]:
library("mlogit")
data("ModeCanada", package = "mlogit")
ModeCanada

载入需要的程序包：dfidx

载入程序包：‘dfidx’

The following object is masked from ‘package:stats’:

    filter



   case   alt choice dist   cost ivt ovt freq income urban noalt
1     1 train      0   83  28.25  50  66    4     45     0     2
2     1   car      1   83  15.77  61   0    0     45     0     2
3     2 train      0   83  28.25  50  66    4     25     0     2
4     2   car      1   83  15.77  61   0    0     25     0     2
5     3 train      0   83  28.25  50  66    4     70     0     2
6     3   car      1   83  15.77  61   0    0     70     0     2
7     4 train      0   83  28.25  50  66    4     70     0     2
8     4   car      1   83  15.77  61   0    0     70     0     2
9     5 train      0   83  28.25  50  66    4     55     0     2
10    5   car      1   83  15.77  61   0    0     55     0     2
11    6 train      0   83  28.25  50  66    4     70     0     2
12    6   car      1   83  15.77  61   0    0     70     0     2
13    7 train      0   83  28.25  50  66    4     15     0     2
14    7   car      1   83  15.77  61   0    0     15     0     2
15    8 train      0   83

## 输出数据

下面进行转化，得到我们想要的数据：

In [ ]:
MC <- dfidx(
  ModeCanada,
  subset = noalt == 4,
  idx = c("case", "alt")  # 编制索引，指定个体标识和方案标识
)
MC

~~~~~~~
 first 10 observations out of 11116 
~~~~~~~
   choice dist   cost ivt ovt freq income urban noalt      idx
1       0  377  58.25 215  74    4     45     0     4 109:rain
2       1  377 142.80  56  85    9     45     0     4  109:air
3       0  377  27.52 301  63    8     45     0     4  109:bus
4       0  377  71.63 262   0    0     45     0     4  109:car
5       0  377  58.25 215  74    4     70     0     4 110:rain
6       1  377 142.80  56  85    9     70     0     4  110:air
7       0  377  27.52 301  63    8     70     0     4  110:bus
8       0  377  71.63 262   0    0     70     0     4  110:car
9       0  377  58.25 215  74    4     35     0     4 111:rain
10      1  377 142.80  56  85    9     35     0     4  111:air

~~~ indexes ~~~~
   case   alt
1   109 train
2   109   air
3   109   bus
4   109   car
5   110 train
6   110   air
7   110   bus
8   110   car
9   111 train
10  111   air
indexes:  1, 2 

## 实战

下面将一个不符合的原始数据转化为我们想要格式的输入数据，再转化为合适的输出格式：

In [4]:
# 导入数据集
library(haven)
library(tidyverse)

mlogit <- read_dta('../econometrics/stata_data/mlogit.dta')
mlogit

# A tibble: 200 × 5
      id female     ice_cream      video puzzle
   <dbl> <dbl+lbl>  <dbl+lbl>      <dbl>  <dbl>
 1    70 0 [male]   1 [chocolate]     47     57
 2   121 1 [female] 2 [vanilla]       63     61
 3    86 0 [male]   3 [strawberry]    58     31
 4   141 0 [male]   3 [strawberry]    53     56
 5   172 0 [male]   2 [vanilla]       53     61
 6   113 0 [male]   2 [vanilla]       63     61
 7    50 0 [male]   2 [vanilla]       53     61
 8    11 0 [male]   2 [vanilla]       39     36
 9    84 0 [male]   2 [vanilla]       58     51
10    48 0 [male]   2 [vanilla]       50     51
# ℹ 190 more rows
# ℹ Use `print(n = ...)` to see more rows

In [5]:
# 该包需要使用长数据格式，先转化数据格式
mlogit |> 
  expand_grid(alt = c(1, 2, 3)) |> 
  mutate(
    choice = if_else(alt == ice_cream, 1, 0)
  ) |> 
  relocate(alt, .after = id) |> 
  relocate(female, .after = ice_cream) |> 
  relocate(choice, .after = ice_cream)

# A tibble: 600 × 7
      id   alt ice_cream      choice female     video puzzle
   <dbl> <dbl> <dbl+lbl>       <dbl> <dbl+lbl>  <dbl>  <dbl>
 1    70     1 1 [chocolate]       1 0 [male]      47     57
 2    70     2 1 [chocolate]       0 0 [male]      47     57
 3    70     3 1 [chocolate]       0 0 [male]      47     57
 4   121     1 2 [vanilla]         0 1 [female]    63     61
 5   121     2 2 [vanilla]         1 1 [female]    63     61
 6   121     3 2 [vanilla]         0 1 [female]    63     61
 7    86     1 3 [strawberry]      0 0 [male]      58     31
 8    86     2 3 [strawberry]      0 0 [male]      58     31
 9    86     3 3 [strawberry]      1 0 [male]      58     31
10   141     1 3 [strawberry]      0 0 [male]      53     56
# ℹ 590 more rows
# ℹ Use `print(n = ...)` to see more rows

In [6]:
mlogit_idx <- mlogit |> 
  expand_grid(alt = c(1, 2, 3)) |> 
  mutate(
    choice = if_else(alt == ice_cream, 1, 0)
  ) |> 
  relocate(alt, .after = id) |> 
  relocate(female, .after = ice_cream) |> 
  relocate(choice, .after = ice_cream) |> 
  dfidx(
    idx = c("id", "alt")
  )

mlogit_idx

# A tibble: 600 × 6
# Index:    200 (id) x 3 (alt)
# Balanced: yes
   idx   ice_cream     choice female     video puzzle
   <idx> <dbl+lbl>      <dbl> <dbl+lbl>  <dbl>  <dbl>
 1 1:1   1 [chocolate]      1 1 [female]    39     41
 2 1:2   1 [chocolate]      0 1 [female]    39     41
 3 1:3   1 [chocolate]      0 1 [female]    39     41
 4 2:1   2 [vanilla]        0 1 [female]    42     41
 5 2:2   2 [vanilla]        1 1 [female]    42     41
 6 2:3   2 [vanilla]        0 1 [female]    42     41
 7 3:1   1 [chocolate]      1 0 [male]      63     56
 8 3:2   1 [chocolate]      0 0 [male]      63     56
 9 3:3   1 [chocolate]      0 0 [male]      63     56
10 4:1   1 [chocolate]      1 1 [female]    39     51
# ℹ 590 more rows
# ℹ Use `print(n = ...)` to see more rows

# 多项Logit模型

In [74]:
fit_mlogit <- mlogit(
  choice ~ 0 | female + video + puzzle,
  data = mlogit_idx,
  reflevel = 1  # 指定参考组
)

summary(fit_mlogit)


Call:
mlogit(formula = choice ~ 0 | female + video + puzzle, data = mlogit_idx, 
    reflevel = 1, method = "nr")

Frequencies of alternatives:choice
    1     2     3 
0.235 0.475 0.290 

nr method
5 iterations, 0h:0m:0s 
g'(-H)^-1g = 2.64E-05 
successive function values within tolerance limits 

Coefficients :
               Estimate Std. Error z-value  Pr(>|z|)    
(Intercept):2 -1.912254   1.127256 -1.6964 0.0898140 .  
(Intercept):3 -5.969578   1.437546 -4.1526 3.287e-05 ***
female:2      -0.816620   0.390981 -2.0886 0.0367400 *  
female:3      -0.849482   0.448212 -1.8953 0.0580570 .  
video:2        0.023565   0.020975  1.1235 0.2612330    
video:3        0.046487   0.025100  1.8521 0.0640176 .  
puzzle:2       0.038924   0.019517  1.9944 0.0461055 *  
puzzle:3       0.081928   0.023834  3.4375 0.0005872 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -194.03
McFadden R^2:  0.078581 
Likelihood ratio test : chisq = 33.095 (p.value = 1.005

# 多项Probit模型

In [10]:
fit_mprobit <- mlogit(
  choice ~ 0 | female + video + puzzle,
  data = mlogit_idx,
  reflevel = 1,  # 指定参考组
  probit = TRUE
)

summary(fit_mprobit)


Call:
mlogit(formula = choice ~ 0 | female + video + puzzle, data = mlogit_idx, 
    reflevel = 1, probit = TRUE)

Frequencies of alternatives:choice
    1     2     3 
0.235 0.475 0.290 

bfgs method
40 iterations, 0h:0m:2s 
g'(-H)^-1g = 9.45E-08 
gradient close to zero 

Coefficients :
                Estimate Std. Error z-value Pr(>|z|)  
(Intercept):2  -1.208479   1.095679 -1.1030   0.2700  
(Intercept):3 -10.363668  39.327822 -0.2635   0.7921  
female:2       -0.513220   0.262189 -1.9574   0.0503 .
female:3       -0.846756   1.946420 -0.4350   0.6635  
video:2         0.012256   0.015594  0.7859   0.4319  
video:3         0.064677   0.221979  0.2914   0.7708  
puzzle:2        0.027317   0.022312  1.2243   0.2208  
puzzle:3        0.119920   0.401844  0.2984   0.7654  
2.3             0.811869   4.648165  0.1747   0.8613  
3.3             3.217398  12.957392  0.2483   0.8039  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -193.83
McFadden R^2:

# 条件Logit模型

In [12]:
# 先导入数据
library(haven)
travel2 <- read_dta("../econometrics/stata_data/travel2.dta")
travel2

# A tibble: 456 × 13
      id mode      train   bus   car  time  invc choice  ttme  invt    gc  hinc
   <dbl> <dbl+lbl> <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl> <dbl>
 1     1 1 [Train]     1     0     0   406    31      0    34   372    71    35
 2     1 2 [Bus]       0     1     0   452    25      0    35   417    70    35
 3     1 3 [Car]       0     0     1   180    10      1     0   180    30    35
 4     2 1 [Train]     1     0     0   398    31      0    44   354    84    30
 5     2 2 [Bus]       0     1     0   452    25      0    53   399    85    30
 6     2 3 [Car]       0     0     1   255    11      1     0   255    50    30
 7     3 1 [Train]     1     0     0   926    98      0    34   892   195    40
 8     3 2 [Bus]       0     1     0   917    53      0    35   882   149    40
 9     3 3 [Car]       0     0     1   720    23      1     0   720   101    40
10     4 1 [Train]     1     0     0   398    26      0    44   354    79    70
# ℹ 446 more rows
#

In [13]:
# 转化为符合要求的格式
travel2_idx <- dfidx(
  travel2,
  idx = c("id", "mode")
)

travel2_idx

# A tibble: 456 × 12
# Index:    152 (id) x 3 (mode)
# Balanced: yes
   idx   train   bus   car  time  invc choice  ttme  invt    gc  hinc psize
   <idx> <dbl> <dbl> <dbl> <dbl> <dbl>  <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1 1:1       1     0     0   406    31      0    34   372    71    35     1
 2 1:2       0     1     0   452    25      0    35   417    70    35     1
 3 1:3       0     0     1   180    10      1     0   180    30    35     1
 4 2:1       1     0     0   398    31      0    44   354    84    30     2
 5 2:2       0     1     0   452    25      0    53   399    85    30     2
 6 2:3       0     0     1   255    11      1     0   255    50    30     2
 7 3:1       1     0     0   926    98      0    34   892   195    40     1
 8 3:2       0     1     0   917    53      0    35   882   149    40     1
 9 3:3       0     0     1   720    23      1     0   720   101    40     1
10 4:1       1     0     0   398    26      0    44   354    79    70     3
# ℹ 446 more rows
#

In [57]:
fit_clogit <- mlogit(
  choice ~  train + bus + time + invc | 0,
  data = travel2_idx
)

summary(fit_clogit)


Call:
mlogit(formula = choice ~ train + bus + time + invc | 0, data = travel2_idx, 
    method = "nr")

Frequencies of alternatives:choice
      1       2       3 
0.41447 0.19737 0.38816 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 1.9E-05 
successive function values within tolerance limits 

Coefficients :
        Estimate Std. Error z-value  Pr(>|z|)    
train  2.6712373  0.4531611  5.8947 3.754e-09 ***
bus    1.4723345  0.4007152  3.6743 0.0002385 ***
time  -0.0191453  0.0024509 -7.8116 5.551e-15 ***
invc  -0.0481658  0.0119516 -4.0301 5.576e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -80.961

In [59]:
fit_clogit <- mlogit(
  choice ~  time + invc | 0,
  data = travel2_idx
)

summary(fit_clogit)


Call:
mlogit(formula = choice ~ time + invc | 0, data = travel2_idx, 
    method = "nr")

Frequencies of alternatives:choice
      1       2       3 
0.41447 0.19737 0.38816 

nr method
5 iterations, 0h:0m:0s 
g'(-H)^-1g = 6.65E-08 
gradient close to zero 

Coefficients :
       Estimate Std. Error z-value  Pr(>|z|)    
time -0.0153450  0.0020370 -7.5333 4.952e-14 ***
invc  0.0025291  0.0070128  0.3606    0.7184    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -106.34

In [58]:
fit_clogit <- mlogit(
  choice ~  train + bus + time + invc | 0 | invt,
  data = travel2_idx
)

summary(fit_clogit)


Call:
mlogit(formula = choice ~ train + bus + time + invc | 0 | invt, 
    data = travel2_idx, method = "nr")

Frequencies of alternatives:choice
      1       2       3 
0.41447 0.19737 0.38816 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 2.32E-05 
successive function values within tolerance limits 

Coefficients :
        Estimate Std. Error z-value  Pr(>|z|)    
train   3.961613   0.828820  4.7798 1.754e-06 ***
bus     3.511116   1.076921  3.2603 0.0011128 ** 
time   -0.060404   0.015154 -3.9861 6.718e-05 ***
invc   -0.044799   0.012952 -3.4588 0.0005427 ***
invt:1  0.044014   0.015962  2.7574 0.0058271 ** 
invt:2  0.042910   0.015836  2.7097 0.0067337 ** 
invt:3  0.044017   0.015660  2.8107 0.0049428 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -75.876

# 混合Logit模型

In [61]:
fit_clogit <- mlogit(
  choice ~  time + invc | hinc + psize,
  data = travel2_idx
)

summary(fit_clogit)


Call:
mlogit(formula = choice ~ time + invc | hinc + psize, data = travel2_idx, 
    method = "nr")

Frequencies of alternatives:choice
      1       2       3 
0.41447 0.19737 0.38816 

nr method
6 iterations, 0h:0m:0s 
g'(-H)^-1g = 3.64E-05 
successive function values within tolerance limits 

Coefficients :
                Estimate Std. Error z-value  Pr(>|z|)    
(Intercept):2 -1.0131755  0.7330291 -1.3822  0.166918    
(Intercept):3 -3.4996387  0.7579665 -4.6171 3.891e-06 ***
time          -0.0185034  0.0025035 -7.3909 1.459e-13 ***
invc          -0.0402791  0.0134851 -2.9869  0.002818 ** 
hinc:2         0.0262667  0.0196277  1.3382  0.180816    
hinc:3         0.0342841  0.0158471  2.1634  0.030508 *  
psize:2       -0.5102610  0.3694765 -1.3810  0.167267    
psize:3        0.0038419  0.3098075  0.0124  0.990106    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -77.505
McFadden R^2:  0.5156 
Likelihood ratio test : chisq = 164.99 (p.value = 

# Hausman检验

In [63]:
travel2 |> 
  distinct(mode)

# A tibble: 3 × 1
  mode     
  <dbl+lbl>
1 1 [Train]
2 2 [Bus]  
3 3 [Car]  

In [65]:
fit_full <- mlogit(
  choice ~  time + invc | hinc + psize,
  data = travel2_idx
) 

In [ ]:
fit_bus <- mlogit(
  choice ~  time + invc | hinc + psize,
  data = travel2_idx,
  alt.subset = c(1, 3)
)

In [68]:
hmftest(fit_full, fit_bus)


	Hausman-McFadden test

data:  travel2_idx
chisq = 51.188, df = 5, p-value = 7.916e-10
alternative hypothesis: IIA is rejected


# 异方差Logit模型

In [69]:
fit_hetlogit <- mlogit(
  choice ~ 0 | female + video + puzzle,
  data = mlogit_idx,
  heterosc = TRUE
)

summary(fit_hetlogit)


Call:
mlogit(formula = choice ~ 0 | female + video + puzzle, data = mlogit_idx, 
    heterosc = TRUE)

Frequencies of alternatives:choice
    1     2     3 
0.235 0.475 0.290 

bfgs method
5 iterations, 0h:0m:0s 
g'(-H)^-1g = 0.0846 
last step couldn't find higher value 

Coefficients :
               Estimate Std. Error z-value Pr(>|z|)   
(Intercept):2 -1.638125   0.847050 -1.9339 0.053123 . 
(Intercept):3 -6.344775   8.053185 -0.7879 0.430779   
female:2      -0.612061   0.299299 -2.0450 0.040857 * 
female:3      -0.715412   0.492907 -1.4514 0.146665   
video:2        0.019957   0.015777  1.2649 0.205894   
video:3        0.047716   0.050544  0.9440 0.345146   
puzzle:2       0.039317   0.014200  2.7689 0.005625 **
puzzle:3       0.082261   0.072606  1.1330 0.257226   
sp.2           0.161323   0.223695  0.7212 0.470801   
sp.3           1.153090   1.794984  0.6424 0.520616   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likelihood: -193.34
McFadden R^2: 

In [75]:
lrtest(fit_hetlogit, fit_mlogit)

Likelihood ratio test

Model 1: choice ~ 0 | female + video + puzzle
Model 2: choice ~ 0 | female + video + puzzle
  #Df  LogLik Df  Chisq Pr(>Chisq)
1  10 -193.34                     
2   8 -194.03 -2 1.3981     0.4971

In [81]:
lrtest(fit_hetlogit)

Likelihood ratio test

Model 1: choice ~ 0 | female + video + puzzle
Model 2: choice ~ 0 | female + video + puzzle
  #Df  LogLik Df  Chisq Pr(>Chisq)
1  10 -193.34                     
2   8 -194.03 -2 1.3981     0.4971

In [79]:
waldtest(fit_hetlogit, fit_mlogit)

Wald test

Model 1: choice ~ 0 | female + video + puzzle
Model 2: choice ~ 0 | female + video + puzzle
  Res.Df Df  Chisq Pr(>Chisq)
1    190                     
2    192 -2 0.6672     0.7164

# 嵌套Logit模型

In [85]:
data("TravelMode", package = "AER")
TravelMode

    individual  mode choice wait vcost travel gcost income size
1            1   air     no   69    59    100    70     35    1
2            1 train     no   34    31    372    71     35    1
3            1   bus     no   35    25    417    70     35    1
4            1   car    yes    0    10    180    30     35    1
5            2   air     no   64    58     68    68     30    2
6            2 train     no   44    31    354    84     30    2
7            2   bus     no   53    25    399    85     30    2
8            2   car    yes    0    11    255    50     30    2
9            3   air     no   69   115    125   129     40    1
10           3 train     no   34    98    892   195     40    1
11           3   bus     no   35    53    882   149     40    1
12           3   car    yes    0    23    720   101     40    1
13           4   air     no   64    49     68    59     70    3
14           4 train     no   44    26    354    79     70    3
15           4   bus     no   53    21  

In [86]:
TravelMode$time <- with(TravelMode, travel + wait)/60
TravelMode$timeair <- with(TravelMode, time * I(mode == "air"))
TravelMode$income <- with(TravelMode, income / 10)
# Hensher and Greene (2002), table 1 p.8-9 model 5
TravelMode$incomeother <- with(TravelMode, ifelse(mode %in% c('air', 'car'), income, 0))
TravelMode

   individual  mode choice wait vcost travel gcost income size      time
1           1   air     no   69    59    100    70    3.5    1  2.816667
2           1 train     no   34    31    372    71    3.5    1  6.766667
3           1   bus     no   35    25    417    70    3.5    1  7.533333
4           1   car    yes    0    10    180    30    3.5    1  3.000000
5           2   air     no   64    58     68    68    3.0    2  2.200000
6           2 train     no   44    31    354    84    3.0    2  6.633333
7           2   bus     no   53    25    399    85    3.0    2  7.533333
8           2   car    yes    0    11    255    50    3.0    2  4.250000
9           3   air     no   69   115    125   129    4.0    1  3.233333
10          3 train     no   34    98    892   195    4.0    1 15.433333
11          3   bus     no   35    53    882   149    4.0    1 15.283333
12          3   car    yes    0    23    720   101    4.0    1 12.000000
13          4   air     no   64    49     68    59 

In [88]:
fit_nestlogit <- mlogit(choice ~ gcost + wait + incomeother, TravelMode,
  nests = list(public = c('train', 'bus'), other = c('car','air')))

summary(fit_nestlogit)


Call:
mlogit(formula = choice ~ gcost + wait + incomeother, data = TravelMode, 
    nests = list(public = c("train", "bus"), other = c("car", 
        "air")))

Frequencies of alternatives:choice
    air   train     bus     car 
0.27619 0.30000 0.14286 0.28095 

bfgs method
12 iterations, 0h:0m:0s 
g'(-H)^-1g = 1.14E-07 
gradient close to zero 

Coefficients :
                    Estimate Std. Error z-value  Pr(>|z|)    
(Intercept):train  0.0056402  0.6363849  0.0089 0.9929285    
(Intercept):bus   -0.7735521  0.8089799 -0.9562 0.3389677    
(Intercept):car   -6.1536851  1.1783823 -5.2221 1.769e-07 ***
gcost             -0.0195488  0.0060331 -3.2402 0.0011943 ** 
wait              -0.1064623  0.0205635 -5.1773 2.252e-07 ***
incomeother        0.4256608  0.1131906  3.7606 0.0001695 ***
iv:public          0.9694958  0.3047966  3.1808 0.0014687 ** 
iv:other           1.7244022  0.5186970  3.3245 0.0008858 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Log-Likeli